In [382]:
import pandas as pd
import numpy as np
import seaborn as sn
import tensorflow as tf
import matplotlib.pyplot as plt
from math import log
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import svm
from sklearn.svm import LinearSVC
from sklearn import neighbors, metrics

from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
from time import sleep
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException 
import re
from datetime import date



## Betting odds Dataframe

Source: <a href="http://tennis-data.co.uk/" target="_blank">http://tennis-data.co.uk/</a>

In [442]:
df_betting=pd.DataFrame()
for i in range(2002,2021):
    
    url="http://tennis-data.co.uk/%d/%d.xlsx" %(i,i)
    df=pd.read_excel(url)
    df_betting= pd.concat([df_betting,df],ignore_index=True)

In [443]:
#Correction for Names not respecting the syntax 
df_betting["Winner"]=df_betting["Winner"].replace({"Kwon S.W.":"Kwon S."})
df_betting["Loser"]=df_betting["Loser"].replace({"Kwon S.W.":"Kwon S."})

Create a datframe with Atp matches since 2002

In [385]:
def checkName(Name):
    if '..' in Name:
        Name=Name.replace('..','.')
    else:
        Name=Name
        
    if '. ' in Name:
        Name=Name.replace('. ','.')
    else:
        Name=Name    
        
    length=len(Name)
    if Name[length-1]!='.':
        Name=Name+'.'
    else:
        Name=Name
    return Name

In [386]:
df_betting_players=pd.DataFrame()
#Keep same syntax for player Name in order to have an unique ID for each player
df_betting_players["Winner"]=df_betting["Winner"].apply(lambda x: checkName(x))
df_betting_players["Loser"]=df_betting["Loser"].apply(lambda x: checkName(x))
df_betting_players["WinnerFName"]=df_betting_players["Winner"].apply(lambda x : " ".join(x.split(" ")[0:-1]))
df_betting_players["LoserFName"]=df_betting_players["Loser"].apply(lambda x : " ".join(x.split(" ")[0:-1]))

In [387]:
df_betting_players.head()

,Winner,Loser,WinnerFName,LoserFName
0,Arazi H.,Kratochvil M.,Arazi,Kratochvil
1,Carlsen K.,Black W.,Carlsen,Black
2,Chela J.I.,Delgado R.,Chela,Delgado
3,Draper S.,Sa A.,Draper,Sa
4,Enqvist T.,Rios M.,Enqvist,Rios


## Players Dataframe

Source: <a href="https://www.kaggle.com/romanzdk/atp-players-overviews" target="_blank">https://www.kaggle.com/romanzdk/atp-players-overviews/</a>

In [388]:
df_players=pd.read_csv('player_overviews.csv')
df_players=df_players.drop(df_players.columns[4:17],axis=1)
df_players=df_players.drop(df_players.columns[-1],axis=1)
df_players.columns=['AtpId','AtpName','FirstName','Name','Height','Handedness']
df_players.head()

,AtpId,AtpName,FirstName,Name,Height,Handedness
0,a001,sadiq-abdullahi,Sadiq,Abdullahi,0.0,NaN
1,a005,nelson-aerts,Nelson,Aerts,188.0,NaN
2,a004,egan-adams,Egan,Adams,178.0,NaN
3,a006,ronald-agenor,Ronald,Agenor,180.0,NaN
4,a007,juan-aguilera,Juan,Aguilera,183.0,NaN


### Change Name syntax to be similar to betting dataframe

For a future merge with the betting dataframe, we need to apply the same syntax to the Name column

In [389]:
def PlayerName(Name,FirstName):
    length= len(Name)
    if "-" in FirstName:
        FirstName=FirstName.split("-")
        player=Name+' '+FirstName[0][0]+'.'+FirstName[1][0]+'.'
    elif " " in FirstName:
        FirstName=FirstName.split()
        if len(FirstName)>1:
            player=Name+' '+FirstName[0][0]+'.'+FirstName[1][0]+'.'
        else:
            player=Name+' '+FirstName[0]+'.'
    elif Name[length-1]=='.':
        player=Name+FirstName[0]+'.'
        
    else:
        player=Name+' '+FirstName[0]+'.'
    return player.title()

In [390]:
df_players['NewName']=df_players.apply(lambda x: PlayerName(x["Name"],x["FirstName"]), axis=1)

Some players have their name and first letter of their firstname similar. We have to distinguished them 

### Update players dataframe

The dataset is not udpated and some current players are missing. The idea is to scrape datas from the https://www.atptour.com/ website.

In [391]:
#Player informations' are accessible through their atp overview webpage. 
#To get this page we need the name and the id given by the website.
#A first scrape on 'https://www.atptour.com/en/rankings/singles/ gives the url of the overview page
#Then for each player, we scrape First name and Last name in their overview webpage and create a dictionnary

def AtpPlayers():
    AtpIdList=[]
    AtpNameList=[]
    FirstNameList=[]
    LastNameList=[]
    ActualRankingList=[]
    
    ##Find last ranking update date
    url='https://www.atptour.com/en/rankings/singles'
    req = Request(url, headers={'User-Agent': ''})
    webpage = urlopen(req).read()
    soup = BeautifulSoup(webpage, 'html.parser')
    update_date=soup.find(attrs={'class':'dropdown-default-label'}).get_text()
    update_date=update_date.replace('.','-').rstrip().lstrip() #clean string 

    ##Access the ranking with the first 700 players
    url='https://www.atptour.com/en/rankings/singles/?rankDate='+update_date+'&countryCode=all&rankRange=0-700'
    req = Request(url, headers={'User-Agent': ''})
    webpage = urlopen(req).read()
    soup1 = BeautifulSoup(webpage, 'html.parser')
    for i in range(0,700):
        
        ##Access the player overview and scrape their AtpId, AtpName, FirstName and Name
        
        url_overview='https://www.atptour.com' + soup1.select('.player-cell a')[i]['href']
        #AtpName and AtpId are present in the href
        AtpName=url_overview.split("/")[5]
        AtpId=url_overview.split("/")[6]
        
        req = Request(url_overview, headers={'User-Agent': 'Mozilla/5.0'})
        webpage = urlopen(req).read()
        soup2 = BeautifulSoup(webpage, 'html.parser')
        try:
            FirstName = soup2.find( attrs={'class': 'first-name'}).get_text()
        except AttributeError:
            FirstName=np.nan
        
        try:
            Name = soup2.find( attrs={'class': 'last-name'}).get_text()
        except AttributeError:
            Name=np.nan
            
        
            
        AtpIdList.append(AtpId)
        AtpNameList.append(AtpName)
        FirstNameList.append(FirstName)
        LastNameList.append(Name)
        ActualRankingList.append(i+1) #create an Actual ranking list useful for validation process
        
        Dictionnary={'AtpId':AtpIdList,'AtpName':AtpNameList,'FirstName':FirstNameList,'Name':LastNameList,'ActualRanking':ActualRankingList}

    return Dictionnary

In [393]:
df_AtpPlayers=AtpPlayers() #get a dictionnary with scraped datas
df_AtpPlayers=pd.DataFrame(df_AtpPlayers) #convert dictionnary into dataframe
df_AtpPlayers['NewName']=df_AtpPlayers.apply(lambda x: PlayerName(x["Name"],x["FirstName"]), axis=1)
#df_AtpPlayers=df_AtpPlayers.drop(["FirstName"],axis=1) #column not needed anymore

In [394]:
df_total_players=pd.concat([df_players, df_AtpPlayers], ignore_index=True) # concat the scraping dataframe with the original one
df_total_players.drop_duplicates(subset ="AtpId",keep = "last", inplace = True) # KEEP UNIQUE VALUES
print(df_total_players.shape)
df_total_players.head()

(11192, 8)


,AtpId,AtpName,FirstName,Name,Height,Handedness,NewName,ActualRanking
0,a001,sadiq-abdullahi,Sadiq,Abdullahi,0.0,NaN,Abdullahi S.,NaN
1,a005,nelson-aerts,Nelson,Aerts,188.0,NaN,Aerts N.,NaN
2,a004,egan-adams,Egan,Adams,178.0,NaN,Adams E.,NaN
3,a006,ronald-agenor,Ronald,Agenor,180.0,NaN,Agenor R.,NaN
4,a007,juan-aguilera,Juan,Aguilera,183.0,NaN,Aguilera J.,NaN


In [395]:
#df_total_players=df_total_players.sort_values(["Name","FirstName"]).reset_index(drop=True)

In [396]:
#df_total_players["Name"]=duplicatedNames(df_total_players)

In [397]:
#df_total_players=df_total_players.drop(["NewName"],axis=1) #column not needed anymore
#df_total_players.head()

In [398]:
print(df_total_players.shape)

(11192, 8)


### Select only commun players

In [399]:
df_selected_players=df_total_players[df_total_players["Name"].isin(df_betting_players["WinnerFName"])|df_total_players["Name"].isin(df_betting_players["LoserFName"])].reset_index(drop=True) 

In [400]:
df_selected_players.head()

,AtpId,AtpName,FirstName,Name,Height,Handedness,NewName,ActualRanking
0,a012,marc-albert,Marc,Albert,0.0,NaN,Albert M.,NaN
1,a021,anand-amritraj,Anand,Amritraj,185.0,NaN,Amritraj A.,NaN
2,a022,vijay-amritraj,Vijay,Amritraj,193.0,NaN,Amritraj V.,NaN
3,a030,roberto-arguello,Roberto,Arguello,173.0,NaN,Arguello R.,NaN
4,a051,juan-carlos-ayala,Juan Carlos,Ayala,NaN,NaN,Ayala J.C.,NaN


In [401]:
def SimilarNames(df):
    column=[df.loc[0,"NewName"]]
    for i in range(1,len(df)):
        
        FirstName=df.loc[i,"FirstName"]
        Name=df.loc[i,"Name"]
        
        j=0
        while df.loc[i,"NewName"]in column:
            
            df.loc[i,"NewName"]=Name+' '+FirstName[0:j+1]+'.'
            j+=1
        
        
        column.append(df.loc[i,"NewName"])
    return column

In [402]:
def UniquePlayers(df_selected_players, df_betting_players):
    
    ##########################################################################################
    ## With this function we want to deal with Players which have the same "NewName".       ##
    ## Some players in the df_betting_players are unique but not in the df_selected_players.##
    ## In this case, the idea is to keep only the player with an actual ranking in order to ##
    ## have a logical further merge. If there is no actual players we keep them all.        ##
    ## The second step will be to use the SimilarNames function to differentiate players    ##
    ## with the same Name in df_selected_players                                            ##
    ##########################################################################################

    newdf=[]
    for i in range(0,len(df_selected_players)):
        
        occurrence=df_selected_players[df_selected_players["NewName"].str.contains(df_selected_players.iloc[i]["NewName"])] # number of occurrence of the NewName in the dataframe
        
        #find unique players in the betting dataframe containing the "NewName" value
        betting_unique=df_betting_players['Winner'][df_betting_players["Winner"].str.contains(df_selected_players.iloc[i]["NewName"])]
        betting_unique=betting_unique.append(df_betting_players['Loser'][df_betting_players["Loser"].str.contains(df_selected_players.iloc[i]["NewName"])])
        betting_unique=betting_unique.unique()
                                             
        if (len(betting_unique)==1) & (occurrence.shape[0]!=1): #if there is a unique player in df_selected_players_betting_payers for multiple players in df_selected_players
            currentPlayers= occurrence[occurrence['ActualRanking'].notna()] #select rows with an 'ActualRanking' value
            if len(currentPlayers)!=0: 
                for j in currentPlayers.index:
                    newdf.append(df_selected_players.iloc[j]) #add only rows with an ActualRanking value
            else:
                newdf.append(df_selected_players.iloc[i])
        else:
            newdf.append(df_selected_players.iloc[i])
            
    newdf=pd.DataFrame(newdf) # convert to pandas dataframe
    newdf.drop_duplicates(subset ="AtpId",keep = "last", inplace = True) # drop duplicated Atp players
    newdf=newdf.sort_values(["Name","FirstName"]).reset_index(drop=True) # sort players by their Name and firstname
    newdf["NewName"]=SimilarNames(newdf) # apply SimilarNames function
    
    return newdf

In [403]:
df_selected_players=UniquePlayers(df_selected_players,df_betting_players)

In [480]:
df_selected_players[df_selected_players["NewName"].str.contains("Bautista")]

,AtpId,AtpName,FirstName,Name,Height,Handedness,NewName,ActualRanking,Id
129,bn54,felix-bautista,Felix,Bautista,NaN,NaN,Bautista F.,NaN,129
130,bd06,roberto-bautista-agut,Roberto,Bautista Agut,183.0,Right-Handed,Bautista Agut R.,11.0,130


In [405]:
print(df_selected_players.shape)

(1943, 8)


Height, Handedness and Backhand are interesting criterias to analyse but we saw in the model prediction notebook that backhand had too many 'Nan' values even after scraping. Thanks to the AtId and AtpNAme we scraped on the website https://www.atptour.com/ 

In [406]:
def scraping(atpId,atpName,height,handedness):
    
    try:
        url_atp = 'https://www.atptour.com/en/players/'+atpName+'/'+atpId+'/overview'
        req = Request(url_atp, headers={'User-Agent': 'Mozilla/5.0'})
        webpage = urlopen(req).read()
        soup = BeautifulSoup(webpage, 'html.parser')

        if ((height!=height) or (height==0.0)): #check NaN value or height equals to 0
            try:
                height = soup.find( attrs={'class': 'table-height-cm-wrapper'}).get_text()
                height= height.replace('(','').replace(')','').replace('cm','')
                if str(height)=='0':
                    height=np.nan
                else:
                    height=height
            except AttributeError:
                height=np.nan
        else:
            height=height

        if handedness!=handedness: #check NaN value
            try:
                plays= soup.find_all( attrs={'class':"table-value"})
                if len(plays)>=2:
                    plays=plays[2].get_text()
                    if len(plays)>1:
                        plays=plays.split(", ")

                        handedness=plays[0].replace("\r","")
                        handedness=handedness.replace("\n","")  
                    else:
                        handedness=plays.replace("\r","")
                        handedness=handedness.replace("\n","")
                    if handedness=='':
                        handedness=np.nan
                    else:
                        handedness=handedness
                else:
                    handedness=np.nan
            except AttributeError:
                handedness=np.nan
        else:
            handedness=handedness

        
    except:
        height=handedness=np.nan
    
    
    
    return height,handedness

In [407]:
def UpdateRanking(atpId,atpName):
    url_atp = 'https://www.atptour.com/en/players/'+atpName+'/'+atpId+'/overview'
    req = Request(url_atp, headers={'User-Agent': 'Mozilla/5.0'})
    webpage = urlopen(req).read()
    soup = BeautifulSoup(webpage, 'html.parser')
    try:
        Ranking = soup.find( attrs={'class': 'data-number'}).get_text()
    except:
        Ranking=np.nan
    try:
        int(Ranking)
        return Ranking
    except ValueError: #For inactive players 
        return np.nan

In [408]:
df_selected_players.dtypes

AtpId             object
AtpName           object
FirstName         object
Name              object
Height           float64
Handedness        object
NewName           object
ActualRanking    float64
dtype: object

In [409]:
df_selected_players[['Height','Handedness']]=df_selected_players.apply(lambda x : scraping(x['AtpId'],x['AtpName'],x['Height'],x['Handedness']),axis=1, result_type="expand")
df_selected_players.head()

,AtpId,AtpName,FirstName,Name,Height,Handedness,NewName,ActualRanking
0,a618,mohammed-abdulla,Mohammed,Abdulla,NaN,NaN,Abdulla M.,NaN
1,a391,maximilian-abel,Maximilian,Abel,NaN,Right-Handed,Abel M.,NaN
2,a389,jose-acasuso,Jose,Acasuso,191,Right-Handed,Acasuso J.,NaN
3,a305,jacob-adaktusson,Jacob,Adaktusson,193,Right-Handed,Adaktusson J.,NaN
4,a310,emin-agaev,Emin,Agaev,NaN,Right-Handed,Agaev E.,NaN


In [410]:
df_selected_players["Height"] = df_selected_players.Height.astype(float) # convert to float

In [411]:
df_selected_players['Id']=df_selected_players.index

In [ ]:
df_copy=df_selected_players.copy()

In [412]:
df_selected_players.head()

,AtpId,AtpName,FirstName,Name,Height,Handedness,NewName,ActualRanking,Id
0,a618,mohammed-abdulla,Mohammed,Abdulla,NaN,NaN,Abdulla M.,NaN,0
1,a391,maximilian-abel,Maximilian,Abel,NaN,Right-Handed,Abel M.,NaN,1
2,a389,jose-acasuso,Jose,Acasuso,191.0,Right-Handed,Acasuso J.,NaN,2
3,a305,jacob-adaktusson,Jacob,Adaktusson,193.0,Right-Handed,Adaktusson J.,NaN,3
4,a310,emin-agaev,Emin,Agaev,NaN,Right-Handed,Agaev E.,NaN,4


## Model Dataframe

### Clean Dataframe

We can notice that depending the year, Betting Broker are not the same. So it's better to use the Min/Max and Average of betting odds. 

In [444]:
#Fill NaN values
df_betting['MaxW']=df_betting['MaxW'].fillna(round(df_betting[['B365W', 'CBW', 'GBW', 'IWW','SBW', 'B&WW', 'EXW', 'PSW', 'UBW', 'LBW', 'SJW']].max(axis=1),2))
df_betting['MaxL']=df_betting['MaxL'].fillna(round(df_betting[['B365L', 'CBL', 'GBL', 'IWL','SBL', 'B&WL', 'EXL', 'PSL', 'UBL', 'LBL', 'SJL']].max(axis=1),2))
df_betting['AvgW']=df_betting['AvgW'].fillna(round(df_betting[['B365W', 'CBW', 'GBW', 'IWW','SBW', 'B&WW', 'EXW', 'PSW', 'UBW', 'LBW', 'SJW']].mean(axis=1),2))
df_betting['AvgL']=df_betting['AvgL'].fillna(round(df_betting[['B365L', 'CBL', 'GBL', 'IWL','SBL', 'B&WL', 'EXL', 'PSL', 'UBL', 'LBL', 'SJL']].mean(axis=1),2))

In [445]:
# drop useless datas
df_betting=df_betting[df_betting['MaxW'].notna() & df_betting['MaxL'].notna()] # select rows where betting odds are present
df_betting=df_betting.drop(df_betting[df_betting["Comment"]!="Completed"].index) #keep only completed matches
df_betting=df_betting.drop(['Comment','Date','ATP','Best of'],axis=1) # drop useless columns
df_betting=df_betting.drop(['B365W', 'CBW', 'GBW', 'IWW','SBW', 'B&WW', 'EXW', 'PSW', 'UBW', 'LBW', 'SJW','B365L', 'CBL', 'GBL', 'IWL','SBL', 'B&WL', 'EXL', 'PSL', 'UBL', 'LBL', 'SJL'],axis=1)
df_betting=df_betting.drop(['W1','L1','W2','L2','W3','L3','W4','L4','W5','L5','Wsets','Lsets','WPts','LPts'],axis=1)
df_betting=df_betting.drop(df_betting[(df_betting["WRank"]=='NR') |  (df_betting["LRank"]=='NR')].index) # drop matches with No ranked players
df_betting=df_betting.dropna()
df_betting["LRank"] = df_betting.LRank.astype(float)# no more 'NR' players so we can convert into float

### Preprocessing data

In [446]:
#Keep same syntax for player Name in order to have an unique ID for each player
df_betting["Winner"]=df_betting["Winner"].apply(lambda x: checkName(x))
df_betting["Loser"]=df_betting["Loser"].apply(lambda x: checkName(x))

In [447]:
#Some Series have been renamed over years
df_betting['Series'].replace({"International Gold":"ATP500","Masters Cup":"ATP Finals","Masters":"ATP Finals"},inplace =True)
#Delete ended white space
df_betting['Location']=df_betting['Location'].apply(lambda x : x.rstrip())

In [448]:
#Rename columns: Winner -> Player1 & Loser -> Player2
df_betting.rename(columns={'Winner':'Player1','Loser':'Player2','WRank':'P1Rank','LRank':'P2Rank','MaxW':'MaxP1','MaxL':'MaxP2','AvgW':'AvgP1','AvgL':'AvgP2'}, inplace=True)
#Create the column for the label prediction: Player1 Win
df_betting['P1Winner']=1 

In the actual format all winners are Player 1, it will be a problem for the model.
So for random rows, Players 1 and Player 2 datas are switched

In [449]:
mask=df_betting.sample(frac=.5).index
df_betting.loc[mask,['Player1','Player2','P1Rank','P2Rank','MaxP1','MaxP2','AvgP1','AvgP2']] = df_betting.loc[mask,['Player2','Player1','P2Rank','P1Rank','MaxP2','MaxP1','AvgP2','AvgP1']].values
df_betting.loc[mask,['P1Winner']]=0

In [451]:
df_betting=df_betting.merge(df_selected_players,left_on='Player1',right_on="NewName", how='left',suffixes=['P2','P1']) #first merge for winner player
df_betting=df_betting.merge(df_selected_players,left_on='Player2',right_on="NewName", how='left',suffixes=['P1','P2'])#second merge for loser player

In [454]:
df_betting=df_betting.drop(['FirstNameP1','FirstNameP2','NameP1','NameP2',"AtpIdP1","AtpIdP2","ActualRankingP1","ActualRankingP2","AtpNameP1","AtpNameP2","Player1","Player2"],axis=1) # keep only the player ID
df_betting=df_betting.dropna() # delete rows with NaN values
df_betting=df_betting.drop_duplicates() #delete duplicated rows

In [456]:
df_betting["HeightP1"] = df_betting.HeightP1.astype(float) # convert to float
df_betting["HeightP2"] = df_betting.HeightP2.astype(float) # convert to float

In [463]:
df_betting.head(10)

,Location,Tournament,Series,Court,Surface,Round,P1Rank,P2Rank,MaxP1,MaxP2,...,AvgP2,P1Winner,HeightP1,HandednessP1,NewNameP1,IdP1,HeightP2,HandednessP2,NewNameP2,IdP2
0,Adelaide,AAPT Championships,International,Outdoor,Hard,1st Round,25.0,46.0,1.60,2.20,...,2.17,1,175.0,Left-Handed,Arazi H.,65.0,185.0,Right-Handed,Kratochvil M.,943.0
1,Adelaide,AAPT Championships,International,Outdoor,Hard,1st Round,309.0,9.0,4.25,1.15,...,1.15,0,196.0,Right-Handed,Fromberg R.,552.0,185.0,Right-Handed,Henman T.,721.0
2,Adelaide,AAPT Championships,International,Outdoor,Hard,1st Round,10.0,57.0,1.36,3.80,...,3.09,0,185.0,Right-Handed,Sampras P.,1494.0,188.0,Right-Handed,Martin T.,1090.0
3,Adelaide,AAPT Championships,International,Outdoor,Hard,1st Round,80.0,93.0,1.50,2.55,...,2.35,1,183.0,Right-Handed,Massu N.,1101.0,183.0,Right-Handed,Savolt A.,1516.0
4,Adelaide,AAPT Championships,International,Outdoor,Hard,1st Round,104.0,53.0,1.50,2.40,...,2.27,1,196.0,Right-Handed,Philippoussis M.,1320.0,191.0,Right-Handed,Boutter J.,201.0
5,Adelaide,AAPT Championships,International,Outdoor,Hard,2nd Round,25.0,150.0,1.45,2.75,...,2.53,1,175.0,Left-Handed,Arazi H.,65.0,185.0,Right-Handed,Mathieu P.H.,1105.0
6,Adelaide,AAPT Championships,International,Outdoor,Hard,2nd Round,24.0,50.0,1.33,3.20,...,2.92,1,191.0,Right-Handed,Enqvist T.,486.0,188.0,Left-Handed,Golmard J.,618.0
7,Adelaide,AAPT Championships,International,Outdoor,Hard,2nd Round,80.0,37.0,2.38,1.60,...,1.54,0,183.0,Right-Handed,Massu N.,1101.0,193.0,Right-Handed,Ljubicic I.,1030.0
8,Adelaide,AAPT Championships,International,Outdoor,Hard,2nd Round,41.0,70.0,1.80,1.90,...,1.86,1,183.0,Right-Handed,Martin A.,1082.0,191.0,Right-Handed,Chela J.I.,308.0
9,Adelaide,AAPT Championships,International,Outdoor,Hard,2nd Round,57.0,89.0,1.40,2.75,...,2.62,1,188.0,Right-Handed,Martin T.,1090.0,191.0,Left-Handed,Llodra M.,1031.0


In [458]:
df_betting["Series"].unique()

array(['International', 'Grand Slam', 'ATP500', 'ATP Finals', 'ATP250',
       'Masters 1000'], dtype=object)

Columns label encoding

In [459]:
label_dictionnary=[] 

In [460]:
def Labelizer(column):
    label=sorted(column.unique())
    dictionnary = { val : idx for idx,val in enumerate(label) }
    label_dictionnary.append(dictionnary) #save label encoder for validation test
    colLab=column.map(dictionnary)
    return colLab

In [ ]:
df_betting=df_betting.apply(lambda x: Labelizer(x) if x.dtype==object else x,axis=0)

## Model

### Multilayers perceptron

In [462]:
X=df_betting.drop(['Tournament','Round','P1Winner','MaxP1','MaxP2',],axis=1)
y=df_betting['P1Winner']


Ml_model=tf.keras.Sequential([
    tf.keras.layers.Dense(16,input_dim=X.shape[1],activation="relu"),
    #tf.keras.layers.Dense(12,activation="relu"),
    tf.keras.layers.Dense(8,activation="relu"),
    #tf.keras.layers.Dense(4,activation="relu"),
    tf.keras.layers.Dense(1,activation="sigmoid"),
])

#Compile the model

Ml_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

#Train the model

history=Ml_model.fit(X,y,epochs=200, batch_size=250,verbose=0)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type float).

## Validation 

The idea is to scrape https://www.oddsportal.com/matches/tennis/ and find ATP matches of the day.

In [540]:
def PlayerSyntax(playerName):
    
    playerName=checkName(playerName)#first syntax check for dots
    
    char_positions=[pos for pos, char in enumerate(playerName) if char == "-"] #array of character position in string
    if char_positions: ##  if not empty array
        for i in char_positions:
            if playerName[i-1].isupper(): #check if the character before is an uppercase
                #convert to list in order to change specific character 
                new=list(playerName)
                new[i]="."
                playerName="".join(new)
                
    if playerName == "Kwon Soonwoo":
        playerName="Kwon S."
    elif playerName == "Ramos A.":
        playerName="Ramos-Vinolas A."
    #elif playerName == "Carreno-Busta P.":
    #    playerName="Carreno Busta P."
        

    return playerName    


In [541]:
def DayMatches():
    #############################################################################
    ## Scrape Location, Players, Odds and Winner of each ATP matches of the day ##
    #############################################################################
    url_odds="https://www.oddsportal.com/matches/tennis/"

    driver = webdriver.Chrome(executable_path=r'/Users/pierremecchia/Desktop/chromedriver')
    driver.get(url_odds)
    sleep(2)

    table=driver.find_elements_by_xpath("//*[@id='table-matches']/table/tbody/tr[contains(@class,'dark')]")

    list_href=[]
    list_location=[]
    list_player1=[]
    list_player2=[]
    list_odd1=[]
    list_odd2=[]
    list_winnerP1=[]    

    ## find all ATP tournament and keep their link
    for row in table:
        if ("ATP" in row.text) and (not "Doubles" in row.text): #only interested in single atp matches
            href=row.find_element_by_xpath('./th[1]/a[2]').get_attribute('href')
            if href not in list_href:
                list_href.append(href)
    driver.quit()
    

    for href in list_href:

        driver = webdriver.Chrome(executable_path=r'/Users/pierremecchia/Desktop/chromedriver')
        driver.get(href)
        sleep(2)
        ##find location of the atp tournament
        h1=driver.find_element_by_xpath('//*[@id="col-content"]/h1').text
        
        #clean location string
        location=h1.replace('Betting Odds','')
        location=location.replace('ATP ','')
        location=re.sub(r'\([^)]*\)', '', location)#delete special characters 
        location=location.rstrip().lstrip() #delete white space at the beginning and end of string
        
        matches=driver.find_elements_by_xpath("//*[@id='tournamentTable']/tbody/tr") #List of matches in the tournament
        doScrap=False
        for match in matches:
            try:
                className=match.get_attribute("class")
                if className=='center nob-border': #create a counter in order to scrape only matches of the day
                    date=match.find_element_by_xpath('./th[1]/span').text
                    if 'Today' in date:
                        doScrap=True
                    else:
                        doScrap=False
                if doScrap==True:
                    if not('dark' or 'nob-border' or 'table-dummyrow')in className:

                        try:
                            if 'deactivate' in className: #for matches completed or currently playing (live matches)
                                players=match.find_element_by_xpath('./td[2]/a').text

                                if players=="": #append for live matches
                                    players=match.find_element_by_xpath('./td[2]/a[2]').text

                                #find odds
                                odd1=match.find_element_by_xpath('./td[4]/a').text
                                odd1_className=match.find_element_by_xpath('./td[4]').get_attribute("class")

                                odd2=match.find_element_by_xpath('./td[5]/a').text
                                odd2_className=match.find_element_by_xpath('./td[5]').get_attribute("class")
                                
                                #find winner of the match
                                winnerP1=match.find_element_by_xpath('./td[3]').text

                                if ("ret" in winnerP1) or ("canc" in winnerP1): #no winner for canceled or retirement matches
                                    winnerP1=np.nan
                                elif (("result-ok" not in odd1_className) & ("result-ok" not in odd2_className)): #match not finished
                                    winnerP1=np.nan
                                elif int(winnerP1[0])<int(winnerP1[2]):
                                    winnerP1=0
                                else:
                                    winnerP1=1
                            else: # for matches not played yet
                                players=match.find_element_by_xpath('./td[2]/a[not(contains(@class,"ico-tv-tournament"))]').text
                                odd1=match.find_element_by_xpath('./td[3]/a').text
                                odd2=match.find_element_by_xpath('./td[4]/a').text
                                winnerP1=np.nan
                            
                            #Clean players names
                            player1=players.split(" - ")[0]
                            player1=PlayerSyntax(player1)
                            player2=players.split(" - ")[1]
                            player2=PlayerSyntax(player2)
                            
                        
                            list_location.append(location)
                            list_player1.append(player1)
                            list_player2.append(player2)
                            list_odd1.append(odd1)
                            list_odd2.append(odd2)
                            list_winnerP1.append(winnerP1)
                            
                        except :
                            continue

            except NoSuchElementException:
                continue
                
        driver.quit()
            
    dictionnary={"Location":list_location,"Player1":list_player1,"Player2":list_player2,"AvgP1":list_odd1,"AvgP2":list_odd2,"WinnerP1":list_winnerP1} #create a dictionnary
    df_matchs=pd.DataFrame(dictionnary) #convert dictionnary into pandas dataframe
    
    return df_matchs
            
    

In [542]:
def TournamentsData(locations_list):
    
    ##################################################################################
    ## Scrape Series, Surface and Court of tournaments present in the locations_list ##
    ##################################################################################
    
    
    url_tourn="https://www.atptour.com/en/tournaments/"

    driver = webdriver.Chrome(executable_path=r'/Users/pierremecchia/Desktop/chromedriver')
    driver.get(url_tourn)
    sleep(4)
    
    #list all tournaments in the current month
    month_tournament=driver.find_elements_by_xpath("//*[@id='contentAccordionWrapper']/div[contains(@class,'expand')][1]/div[2]/div/table/tbody/tr")

    list_series=[]
    list_court=[]
    list_location=[]
    list_surface=[]
    
    for tournament in month_tournament:
        try:
            #scrape location for each tournament in the list
            location=tournament.find_element_by_xpath('./td[2]/span[1]').text 
            town=location.split(",")[0]
            
            if town in locations_list: #identify tournaments that we have to scrape
                try:
                    #define the serie of the tournament thanks to the image source
                    series=tournament.find_element_by_xpath('./td[1]/img').get_attribute("src")
                    if "250.png" in series:
                        series="ATP250"
                    elif "500.png" in series:
                        series="ATP500"
                    elif "1000.png" in series:
                        series="Masters 1000"
                    elif "grandslam.png" in series:
                        series="Grand Slam"
                    elif "finals.svg" in series:
                        series="ATP Finals"
                    else:
                        series=np.nan
                except:
                    series=np.nan
                try:
                    #scrape court and surface of the tournament
                    playground=tournament.find_element_by_xpath('./td[3]/table/tbody/tr/td[2]/div/div').text
                    playground=playground.split(" ")
                    court=playground[0]
                    surface=playground[1]
                    
                except:
                    court=np.nan
                    surface=np.nan
                    
                    
                list_series.append(series)
                list_surface.append(surface)
                list_court.append(court)
                list_location.append(town)
        except:
            continue
    driver.quit()

    loc_dict={"Location":list_location,"Series":list_series,"Court":list_court,"Surface":list_surface} #create a dictionnary
    df_tournaments=pd.DataFrame(loc_dict) # convert dictionnary into pandas dataframe
    
    return df_tournaments

In [543]:
df_matchs=DayMatches() #Scrape atp matches of the day
locations_list=df_matchs['Location'].unique() #location list of tournaments of the day
df_tournaments=TournamentsData(locations_list) # scrape additionnals datas of the tournaments of the day

In [544]:
df_tournaments.head()

,Location,Series,Court,Surface
0,Monte Carlo,Masters 1000,Outdoor,Clay


In [545]:
df_validation=df_matchs.merge(df_tournaments,on="Location")
df_validation.head(20)

,Location,Player1,Player2,AvgP1,AvgP2,WinnerP1,Series,Court,Surface
0,Monte Carlo,Dimitrov G.,Chardy J.,1.39,3.04,1,Masters 1000,Outdoor,Clay
1,Monte Carlo,Fognini F.,Thompson J.,1.19,4.82,1,Masters 1000,Outdoor,Clay
2,Monte Carlo,Millman J.,Garin C.,3.93,1.26,0,Masters 1000,Outdoor,Clay
3,Monte Carlo,Rublev A.,Caruso S.,1.08,8.67,1,Masters 1000,Outdoor,Clay
4,Monte Carlo,Djokovic N.,Sinner J.,1.34,3.33,1,Masters 1000,Outdoor,Clay
5,Monte Carlo,Schwartzman D.,Ruud C.,1.53,2.52,0,Masters 1000,Outdoor,Clay
6,Monte Carlo,Goffin D.,Cecchinato M.,1.55,2.44,1,Masters 1000,Outdoor,Clay
7,Monte Carlo,Krajinovic F.,Londero J.I.,1.18,5.01,1,Masters 1000,Outdoor,Clay
8,Monte Carlo,Popyrin Al.,Pouille L.,1.62,2.30,0,Masters 1000,Outdoor,Clay
9,Monte Carlo,Evans D.,Hurkacz H.,2.54,1.53,1,Masters 1000,Outdoor,Clay


In [546]:
def PlayerNotFound(Player,df_selected_players):
    if Player not in (df_selected_players["NewName"]):
        firstName=Player.split(" ")[-1]
        Name=" ".join(Player.split(" ")[0:-1])

        if df_selected_players["NewName"].str.contains(Name+" "+firstName[0]).any():
            similarPlayer=[]
            i=0
            while len(similarPlayer) != 1:
                similarPlayer=df_selected_players["NewName"][df_selected_players["NewName"].str.contains(Name+" "+firstName[0:i+1])].reset_index(drop=True)
                i+=1
                if i==len(firstName):
                    similarPlayer=Player
                    break
            Player=similarPlayer
        elif df_selected_players["NewName"].str.contains(Name.replace("-"," ")+" "+firstName[0]).any():
            similarPlayer=[]
            i=0
            while len(similarPlayer) != 1:
                similarPlayer=df_selected_players["NewName"][df_selected_players["NewName"].str.contains(Name.replace("-"," ")+" "+firstName[0:i+1])].reset_index(drop=True)
                i+=1
                if i==len(firstName):
                    similarPlayer=Player
                    break
            Player=similarPlayer
        else:
            Player=Player
        
    return Player

In [547]:
df_validation['Player1']=df_validation.apply(lambda x: PlayerNotFound(x["Player1"],df_selected_players),axis=1)
df_validation['Player2']=df_validation.apply(lambda x: PlayerNotFound(x["Player2"],df_selected_players),axis=1)
df_validation.head(20)

,Location,Player1,Player2,AvgP1,AvgP2,WinnerP1,Series,Court,Surface
0,Monte Carlo,Dimitrov G.,Chardy J.,1.39,3.04,1,Masters 1000,Outdoor,Clay
1,Monte Carlo,Fognini F.,Thompson J.,1.19,4.82,1,Masters 1000,Outdoor,Clay
2,Monte Carlo,Millman J.,Garin C.,3.93,1.26,0,Masters 1000,Outdoor,Clay
3,Monte Carlo,Rublev A.,Caruso S.,1.08,8.67,1,Masters 1000,Outdoor,Clay
4,Monte Carlo,Djokovic N.,Sinner J.,1.34,3.33,1,Masters 1000,Outdoor,Clay
5,Monte Carlo,Schwartzman D.,Ruud C.,1.53,2.52,0,Masters 1000,Outdoor,Clay
6,Monte Carlo,Goffin D.,Cecchinato M.,1.55,2.44,1,Masters 1000,Outdoor,Clay
7,Monte Carlo,Krajinovic F.,Londero J.I.,1.18,5.01,1,Masters 1000,Outdoor,Clay
8,Monte Carlo,Popyrin A.,Pouille L.,1.62,2.30,0,Masters 1000,Outdoor,Clay
9,Monte Carlo,Evans D.,Hurkacz H.,2.54,1.53,1,Masters 1000,Outdoor,Clay


In [548]:
df_validation=df_validation.merge(df_selected_players,left_on='Player1',right_on="NewName", how='left',suffixes=['P2','P1']) #first merge for winner player
df_validation=df_validation.merge(df_selected_players,left_on='Player2',right_on="NewName", how='left',suffixes=['P1','P2'])#second merge for loser player

In [549]:
df_validation=df_validation.drop(['FirstNameP1','FirstNameP2','NameP1','NameP2',"AtpIdP1","AtpIdP2","AtpNameP1","AtpNameP2"],axis=1) # drop useless features 

In [550]:
df_validation.head(20)

,Location,Player1,Player2,AvgP1,AvgP2,WinnerP1,Series,Court,Surface,HeightP1,HandednessP1,NewNameP1,ActualRankingP1,IdP1,HeightP2,HandednessP2,NewNameP2,ActualRankingP2,IdP2
0,Monte Carlo,Dimitrov G.,Chardy J.,1.39,3.04,1,Masters 1000,Outdoor,Clay,191.0,Right-Handed,Dimitrov G.,17.0,432,188.0,Right-Handed,Chardy J.,53.0,306
1,Monte Carlo,Fognini F.,Thompson J.,1.19,4.82,1,Masters 1000,Outdoor,Clay,178.0,Right-Handed,Fognini F.,18.0,542,183.0,Right-Handed,Thompson J.,63.0,1712
2,Monte Carlo,Millman J.,Garin C.,3.93,1.26,0,Masters 1000,Outdoor,Clay,183.0,Right-Handed,Millman J.,44.0,1140,185.0,Right-Handed,Garin C.,24.0,584
3,Monte Carlo,Rublev A.,Caruso S.,1.08,8.67,1,Masters 1000,Outdoor,Clay,188.0,Right-Handed,Rublev A.,8.0,1463,185.0,Right-Handed,Caruso S.,89.0,287
4,Monte Carlo,Djokovic N.,Sinner J.,1.34,3.33,1,Masters 1000,Outdoor,Clay,188.0,Right-Handed,Djokovic N.,1.0,436,188.0,Right-Handed,Sinner J.,22.0,1580
5,Monte Carlo,Schwartzman D.,Ruud C.,1.53,2.52,0,Masters 1000,Outdoor,Clay,170.0,Right-Handed,Schwartzman D.,9.0,1527,183.0,Right-Handed,Ruud C.,27.0,1474
6,Monte Carlo,Goffin D.,Cecchinato M.,1.55,2.44,1,Masters 1000,Outdoor,Clay,180.0,Right-Handed,Goffin D.,15.0,612,185.0,Right-Handed,Cecchinato M.,92.0,289
7,Monte Carlo,Krajinovic F.,Londero J.I.,1.18,5.01,1,Masters 1000,Outdoor,Clay,185.0,Right-Handed,Krajinovic F.,37.0,942,180.0,Right-Handed,Londero J.I.,94.0,1035
8,Monte Carlo,Popyrin A.,Pouille L.,1.62,2.30,0,Masters 1000,Outdoor,Clay,196.0,Right-Handed,Popyrin A.,83.0,1345,185.0,Right-Handed,Pouille L.,86.0,1350
9,Monte Carlo,Evans D.,Hurkacz H.,2.54,1.53,1,Masters 1000,Outdoor,Clay,175.0,Right-Handed,Evans D.,33.0,503,196.0,Right-Handed,Hurkacz H.,16.0,755


In [ ]:
#df_active_players=df_selected_players.dropna().reset_index(drop=True)
#df_active_players.head()

In [ ]:
#df_active_players[df_active_players["Name"].str.contains("Evans")]
#df_AtpPlayers[df_AtpPlayers["AtpId"]=="mh30"]
#df_active_players[df_active_players["AtpId"]=="mh30"]